In [20]:

import pandas as pd
import xlrd
import mip
import datetime

# costanti
N_TRATTE = 156
N_LUOGHI = 16


# lettura files
Tratte = pd.read_excel('file_di_dati.xlsx', sheet_name="porzioni di viaggio")
t_v = pd.read_excel('file_di_dati.xlsx', sheet_name="tempi di percorrenza a vuoto")

# metto tutte le colonne in un array
passeggeri = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='B', skiprows=[0])
extra_passengers = [passeggeri.values.tolist()]


init_stop = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='C', skiprows=[0])
inital_stop = [init_stop.values.tolist()]


fin_stop = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='D', skiprows=[0])
final_stop = [fin_stop.values.tolist()]

init_time = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='E', skiprows=[0]) 
initial_time = [init_time.values.tolist()]

fin_time = pd.read_excel("file_di_dati.xlsx",'porzioni di viaggio', usecols='F', skiprows=[0]) 
final_time = [fin_time.values.tolist()]

#tempi a vuoto
tempi_vuoto = (t_v.values.tolist())
#print(tempi_vuoto)

# funzione per convertie i tempi
epoch = datetime.datetime.utcfromtimestamp(0)
def unix_time_sec(dt):
    return (dt - epoch).total_seconds()



In [26]:
# uniformo i nomi tra tempi di percorrenza e tempi a vuoto
#print(len(init_stop))
#for i in range(0,len(init_stop)):
#    initial_stop[i] = initial_stop[i].strip()
#    final_stop[i] = final_stop[i].strip()

#ribalto la matrice per potere scrivere tratte_vuoto[i][j]
for i in range(N_LUOGHI+1):
    for j in range(N_LUOGHI+1):
        if i>j:
            tempi_vuoto[j][i]=tempi_vuoto[i][j]
        if i == j:
            tempi_vuoto[i][j]=0

luoghi = tempi_vuoto[0][:]
luoghi.pop(0)
print(tempi_vuoto)
# usiamo un dizionario in cui ogni nome corrisponde a un indice
luoghi_to_indici = {luoghi[i]:i for i in range(N_LUOGHI)}



[[0, 'stazione', 'kennedy', 'barriera', 'canapa', 'ospedale', 'medaglie d’oro', 'p. dante', 'pontegradella itis', 'barlaam', 'cavour giardini', 'santa maria maddalena', 'autostazione', 'corti di medoro', 'giovecca montebello', 'barco', 'malborghetto'], ['stazione', 0, 10, 3, 10, 20, 10, 10, 12, 10, 5, 20, 5, 15, 8, 15, 25], ['kennedy', 10, 0, 10, 15, 15, 5, 15, 7, 5, 5, 25, 10, 5, 7, 20, 25], ['barriera', 3, 10, 0, 10, 20, 8, 10, 10, 10, 3, 20, 5, 15, 5, 15, 25], ['canapa', 10, 15, 10, 0, 25, 10, 5, 10, 12, 12, 18, 10, 15, 20, 3, 15], ['ospedale', 20, 15, 20, 25, 0, 15, 25, 20, 25, 20, 30, 20, 20, 15, 25, 30], ['medaglie d’oro', 10, 5, 8, 10, 15, 0, 10, 3, 15, 5, 25, 10, 12, 3, 12, 20], ['p. dante', 10, 15, 10, 5, 25, 10, 0, 8, 10, 10, 20, 12, 15, 10, 8, 13], ['pontegradella itis', 12, 7, 10, 10, 20, 3, 8, 0, 17, 7, 25, 12, 15, 5, 12, 20], ['barlaam', 10, 5, 10, 12, 25, 15, 10, 17, 0, 12, 25, 5, 3, 10, 20, 25], ['cavour giardini', 5, 5, 3, 12, 20, 5, 10, 7, 12, 0, 25, 7, 7, 5, 20, 20],

In [4]:
# i nodi sono per ogni tratta 2
# in particolare uno è la partenza e uno l' arrivo
# definisco tutti i nodi, li collgo in base alla tratta 
# definisco anche il nodo pozzo e il nodo sorgente

s = 0
t = n_tratte+1

nodi_partenze = []

for i in range(1,n_tratte+1):
    nodi_partenze.append(i)

nodi_arrivi = []

for i in range(1,n_tratte+1):
    nodi_arrivi.append(i)


#print(nodi_arrivi)



Definiamo archi, nodi, flussi e costi

i nodi sono per ogni tratta 2
in particolare uno è la partenza e uno l' arrivo
definisco tutti i nodi, li collgo in base alla tratta 
definisco anche il nodo pozzo e il nodo sorgente

In [ ]:


A=[[]*]

In [5]:
A = []

# arco s
s = 0
for j in range (n_tratte):
    A.append((s,j))
    
# archi compatibilità
for i in range(n_tratte):
    indice_luogo_corrente = luoghi_to_indici[final_stop[i][0]]
    for j in range(n_tratte):
        indice_luogo_desiderato = luoghi_to_indici[initial_stop[j][0]]
        if i!= j and initial_time[j][0] <= final_time[i][0] + tempi_vuoto[indice_luogo_corrente+1][indice_luogo_desiderato+1]:
            A.append((i+1,j+1))

# archi bilancio
#arco t
t = n_tratte+2
for i in range(n_tratte):
    A.append((i,t))

#print(A)